In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd 
import json
import random
from PIL import Image
import PIL.ImageOps    

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   

## Preprocessing

In [2]:
!mkdir dataset
!tar -xf ssbi-dataset-v1.0.0.zip -C dataset

A subdirectory or file dataset already exists.


In [2]:
train_json_path = "dataset/annotations/instances_train.json"
val_json_path = "dataset/annotations/instances_val.json"

In [3]:
#given a path to a coco json return a df containing filename, category_id, bounding box 
def coco_to_df(path):
    with open(path) as f:
        j = json.load(f)

    #put annotations and images into their own dataframs 
    annotations = pd.DataFrame(j['annotations'])
    images = pd.DataFrame(j['images'])

    #merge the two to get all important info and filter for only categories 6 or 7 forged or genuine 
    df1 = annotations[["image_id", "category_id", "bbox"]]
    df2 = images[["id", "file_name"]]
    df2 = df2.rename(columns={"id": "image_id"})
    final = df1.merge(df2, on='image_id')

    final = final[final['category_id'].isin([6, 7])].reset_index(drop=True)
    final['category_id'] = final['category_id'].replace(6, 1)
    final['category_id'] = final['category_id'].replace(7, 0)
    return final

In [7]:
train_df = coco_to_df(train_json_path)
val_df = coco_to_df(val_json_path)

display(train_df)
display(val_df)
display(len(train_df))
display(len(val_df))

,image_id,category_id,bbox,file_name
0,1,1,"[2574, 856, 284, 307]",check_009_001_F1_0.jpg
1,2,1,"[1671, 599, 166, 180]",check_008_001_F1_0.jpg
2,5,1,"[746, 262, 98, 106]",check_007_001_F1_0.jpg
3,7,1,"[655, 239, 92, 85]",check_004_001_F1_1.jpg
4,9,1,"[717, 262, 114, 106]",check_007_001_F1_1.jpg
...,...,...,...,...
3047,4352,0,"[818, 331, 109, 70]",check_012_019_G3_6.jpg
3048,4353,0,"[1036, 417, 149, 96]",check_016_019_G3_6.jpg
3049,4355,0,"[720, 242, 148, 95]",check_020_019_G3_6.jpg
3050,4357,0,"[872, 261, 93, 70]",check_017_019_G3_7.jpg


,image_id,category_id,bbox,file_name
0,3,1,"[1592, 410, 165, 179]",check_001_001_F1_0.jpg
1,4,1,"[1212, 493, 174, 188]",check_002_001_F1_0.jpg
2,6,1,"[1857, 553, 224, 207]",check_010_001_F1_1.jpg
3,8,1,"[933, 493, 203, 188]",check_002_001_F1_1.jpg
4,10,1,"[1141, 563, 194, 180]",check_005_001_F1_1.jpg
...,...,...,...,...
1303,4349,0,"[698, 294, 55, 39]",check_019_019_G3_5.jpg
1304,4353,0,"[547, 244, 95, 61]",check_018_019_G3_6.jpg
1305,4355,0,"[696, 242, 126, 95]",check_020_019_G3_7.jpg
1306,4358,0,"[747, 331, 93, 70]",check_012_019_G3_7.jpg


3052

1308

In [4]:
#takes in a dataframe of image and will crop and send back image with id (tensor, id)
class SSBI_Dataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df
        self.path = path 
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        image = Image.open(os.path.join(self.path, row["file_name"])).convert("RGB")

        #crop based on bounding box
        x, y, w, h = row['bbox']
        image = image.crop([x, y, x+w, y+h])

        label = int(row["category_id"])
    
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    
    transforms.ColorJitter(brightness=0.1, contrast=0.1), 
    transforms.RandomAffine(
        degrees=5, 
        translate=(0.05, 0.05),
        scale=(0.95, 1.05)       
    ),

    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406], 
        [0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [8]:
train_dataset = SSBI_Dataset(train_df, "dataset/train", train_transform)
val_dataset = SSBI_Dataset(val_df, "dataset/val", val_transform)

In [9]:
print(len(train_dataset))
print(len(val_dataset))

3052
1308


In [10]:
image, label = train_dataset[0]
print(image, label)

tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ...,  1.0844,  1.0673,  1.0673],
         [-2.1179, -2.1179, -2.1179,  ...,  1.0159,  1.0331, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ...,  1.3782,  1.3606,  1.3256],
         [-2.0357, -2.0357, -2.0357,  ...,  1.2731,  1.2906, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

## Train ResNet 

In [11]:
dataloaders = {'train' : torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0), 
               'val' : torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=0)}

dataset_sizes = {'train' : len(train_dataset), 'val' : len(val_dataset)}

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [12]:
#from pytorch documentation
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [13]:
#from pytorch documentation
def train_model(model, criterion, optimizer, scheduler, patience=7, num_epochs=25):
    since = time.time()
    best_val_loss = float('inf') 
    patience_counter = 0 

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0
        stop_training_flag = False

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val':
                    
                    if epoch_acc > best_acc:
                        best_acc = epoch_acc
                        torch.save(model.state_dict(), best_model_params_path)
                        
                    if epoch_loss < best_val_loss:
                        best_val_loss = epoch_loss
                        patience_counter = 0 
                    else:
                        patience_counter += 1 

                    if patience_counter >= patience:
                        print('\n' + '='*20)
                        print(f'EARLY STOPPING: Validation loss did not improve for {patience} epochs.')
                        print(f'Stopping at epoch {epoch}.')
                        print('='*20)
                        stop_training_flag = True
                        break 

            print()
            if stop_training_flag:
                break

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [14]:
model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [15]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=50)
torch.save(model_conv.state_dict(), 'resnet_best_weights.pth')

Epoch 0/49
----------
train Loss: 0.6402 Acc: 0.7274
val Loss: 0.4829 Acc: 0.7607

Epoch 1/49
----------
train Loss: 0.6134 Acc: 0.7418
val Loss: 0.5019 Acc: 0.7798

Epoch 2/49
----------
train Loss: 0.6942 Acc: 0.7333
val Loss: 0.4784 Acc: 0.7821

Epoch 3/49
----------
train Loss: 0.5731 Acc: 0.7507
val Loss: 0.6798 Acc: 0.6735

Epoch 4/49
----------
train Loss: 0.5863 Acc: 0.7526
val Loss: 0.5717 Acc: 0.7271

Epoch 5/49
----------
train Loss: 0.5824 Acc: 0.7625
val Loss: 0.5635 Acc: 0.7202

Epoch 6/49
----------
train Loss: 0.5716 Acc: 0.7556
val Loss: 1.0606 Acc: 0.5291

Epoch 7/49
----------
train Loss: 0.4934 Acc: 0.7720
val Loss: 0.5880 Acc: 0.7156

Epoch 8/49
----------
train Loss: 0.4460 Acc: 0.7916
val Loss: 0.5073 Acc: 0.7546

Epoch 9/49
----------
train Loss: 0.4705 Acc: 0.7798
val Loss: 0.5526 Acc: 0.7294

EARLY STOPPING: Validation loss did not improve for 7 epochs.
Stopping at epoch 9.

Training complete in 12m 38s
Best val Acc: 0.782110


In [17]:
best_weights_path = 'resnet_best_weights.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv.load_state_dict(torch.load(best_weights_path, map_location=device, weights_only=False), strict=False)

for param in model_conv.parameters():
    param.requires_grad = False

for param in model_conv.layer4.parameters():
    param.requires_grad = True

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

#layer4 and last layer
optimizer_conv = optim.SGD(
    [
        {'params': model_conv.fc.parameters(), 'lr': 0.001},  
        
        {'params': model_conv.layer4.parameters(), 'lr': 0.0001} 
    ], 
    momentum=0.9
)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [18]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=100)
torch.save(model_conv.state_dict(), 'resnet_best_weights_2.pth')

Epoch 0/99
----------
train Loss: 0.4495 Acc: 0.8021
val Loss: 0.4144 Acc: 0.8012

Epoch 1/99
----------
train Loss: 0.3540 Acc: 0.8470
val Loss: 0.5462 Acc: 0.7355

Epoch 2/99
----------
train Loss: 0.2951 Acc: 0.8732
val Loss: 0.3753 Acc: 0.8303

Epoch 3/99
----------
train Loss: 0.2520 Acc: 0.8978
val Loss: 0.3066 Acc: 0.8769

Epoch 4/99
----------
train Loss: 0.2391 Acc: 0.9050
val Loss: 0.4296 Acc: 0.7974

Epoch 5/99
----------
train Loss: 0.2081 Acc: 0.9223
val Loss: 0.2408 Acc: 0.9106

Epoch 6/99
----------
train Loss: 0.1951 Acc: 0.9289
val Loss: 0.2536 Acc: 0.8998

Epoch 7/99
----------
train Loss: 0.1831 Acc: 0.9348
val Loss: 0.2039 Acc: 0.9213

Epoch 8/99
----------
train Loss: 0.1858 Acc: 0.9292
val Loss: 0.1542 Acc: 0.9411

Epoch 9/99
----------
train Loss: 0.1834 Acc: 0.9325
val Loss: 0.2363 Acc: 0.9037

Epoch 10/99
----------
train Loss: 0.1695 Acc: 0.9384
val Loss: 0.2212 Acc: 0.9167

Epoch 11/99
----------
train Loss: 0.1873 Acc: 0.9364
val Loss: 0.3342 Acc: 0.8578

Ep

In [19]:
best_weights_path = 'resnet_best_weights_2.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv.load_state_dict(torch.load(best_weights_path, map_location=device, weights_only=False), strict=False)

for param in model_conv.parameters():
    param.requires_grad = False

for param in model_conv.layer4.parameters():
    param.requires_grad = True

for param in model_conv.layer3.parameters():
    param.requires_grad = True

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

#layer4 and last layer
optimizer_conv = optim.SGD(
    [
        {'params': model_conv.fc.parameters(), 'lr': 0.001},  
        
        {'params': model_conv.layer4.parameters(), 'lr': 0.0001},

        {'params': model_conv.layer3.parameters(), 'lr': 0.00001}
    ], 
    momentum=0.9
)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [20]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=100)
torch.save(model_conv.state_dict(), 'resnet_best_weights_3.pth')

Epoch 0/99
----------
train Loss: 0.1807 Acc: 0.9305
val Loss: 0.1704 Acc: 0.9320

Epoch 1/99
----------
train Loss: 0.1645 Acc: 0.9440
val Loss: 0.1881 Acc: 0.9274

Epoch 2/99
----------
train Loss: 0.1480 Acc: 0.9466
val Loss: 0.2060 Acc: 0.9136

Epoch 3/99
----------
train Loss: 0.1406 Acc: 0.9554
val Loss: 0.1265 Acc: 0.9495

Epoch 4/99
----------
train Loss: 0.1362 Acc: 0.9554
val Loss: 0.2546 Acc: 0.9052

Epoch 5/99
----------
train Loss: 0.1342 Acc: 0.9545
val Loss: 0.1662 Acc: 0.9388

Epoch 6/99
----------
train Loss: 0.1298 Acc: 0.9581
val Loss: 0.2067 Acc: 0.9174

Epoch 7/99
----------
train Loss: 0.1089 Acc: 0.9663
val Loss: 0.1946 Acc: 0.9235

Epoch 8/99
----------
train Loss: 0.1113 Acc: 0.9636
val Loss: 0.1648 Acc: 0.9358

Epoch 9/99
----------
train Loss: 0.0984 Acc: 0.9715
val Loss: 0.1783 Acc: 0.9289

Epoch 10/99
----------
train Loss: 0.1071 Acc: 0.9699
val Loss: 0.1954 Acc: 0.9243

EARLY STOPPING: Validation loss did not improve for 7 epochs.
Stopping at epoch 10.

T

In [21]:
best_weights_path = 'resnet_best_weights_3.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv.load_state_dict(torch.load(best_weights_path, map_location=device, weights_only=False), strict=False)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

In [22]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=150)
torch.save(model_conv.state_dict(), 'resnet_best_weights_4.pth')

Epoch 0/149
----------
train Loss: 0.3510 Acc: 0.8834
val Loss: 0.0370 Acc: 0.9901

Epoch 1/149
----------
train Loss: 0.0719 Acc: 0.9780
val Loss: 0.0213 Acc: 0.9992

Epoch 2/149
----------
train Loss: 0.0388 Acc: 0.9885
val Loss: 0.0081 Acc: 0.9992

Epoch 3/149
----------
train Loss: 0.0486 Acc: 0.9882
val Loss: 0.0048 Acc: 0.9992

Epoch 4/149
----------
train Loss: 0.0567 Acc: 0.9866
val Loss: 0.0018 Acc: 0.9992

Epoch 5/149
----------
train Loss: 0.0198 Acc: 0.9971
val Loss: 0.0028 Acc: 0.9992

Epoch 6/149
----------
train Loss: 0.0283 Acc: 0.9961
val Loss: 0.0013 Acc: 1.0000

Epoch 7/149
----------
train Loss: 0.0195 Acc: 0.9964
val Loss: 0.0023 Acc: 1.0000

Epoch 8/149
----------
train Loss: 0.0214 Acc: 0.9944
val Loss: 0.0191 Acc: 0.9954

Epoch 9/149
----------
train Loss: 0.0157 Acc: 0.9954
val Loss: 0.0017 Acc: 1.0000

Epoch 10/149
----------
train Loss: 0.0050 Acc: 0.9990
val Loss: 0.0010 Acc: 1.0000

Epoch 11/149
----------
train Loss: 0.0088 Acc: 0.9984
val Loss: 0.0020 Acc